In [ ]:
import pandas as pd
from langchain_community.vectorstores import FAISS
from langchain_gigachat import  GigaChatEmbeddings, GigaChat
from langchain.schema import Document
from datasets import Dataset
import pickle
import ragas
from ragas import evaluate
from ragas.metrics import (faithfulness, 
                           context_precision, 
                           context_recall, 
                           answer_correctness, 
                           answer_relevancy, 
                           noise_sensitivity,
                           faithfulness_ru, 
                           answer_correctness_ru, 
                           answer_correctness_devide_ru, 
                           answer_relevancy_ru)
from ragas.run_config import RunConfig

cert_file='C:\\Users/22589910/Desktop/work/certs/our_certs/chain_ift_pem.pem'  
key_file='C:\\Users/22589910/Desktop/work/certs/our_certs/CI02851867-IFT-RISKASSISTANT2.key'  
%load_ext autoreload
%autoreload 2

In [ ]:
embedder = GigaChatEmbeddings(
            base_url='https://gigachat-ift.sberdevices.delta.sbrf.ru/v1',
            model= 'Embeddings',
            cert_file=cert_file,
            key_file=key_file,
            verify_ssl_certs=False,

)
llm = GigaChat(
            base_url='https://gigachat-ift.sberdevices.delta.sbrf.ru/v1',
            model = 'GigaChat-2-MAX',
            cert_file=cert_file,
            key_file=key_file, 
            verify_ssl_certs=False,
            temperature=0.0,
            max_tokens=32000,
            profanity_check=False,
)


In [ ]:
# from ragas.llms.base import BaseRagasLLM, LangchainLLMWrapper

# llm = LangchainLLMWrapper(llm)

In [ ]:
bench_df = pd.read_excel('after_adding_doc_name_.xlsx')
bench_df = bench_df[bench_df['docs_bucket'] == 'group standards']
queries = list(bench_df['Вопрос'])
answers = list(bench_df['Ответ'])

In [ ]:
def create_dataset(res):
    df = pd.DataFrame({
        'user_input': queries,
        'retrieved_contexts': [r[1] for r in res if r],
        'response': [r[0] for r in res if r],
        'reference': answers
    })
    return Dataset.from_pandas(df)

In [ ]:
config = RunConfig(max_workers=8, 
                   max_retries=1, 
                   max_wait=300, 
                   timeout=400)

In [ ]:
def evaluate_all_metrics(dataset, llm, config):
    metrics = evaluate(dataset=dataset, 
         metrics=[
                # faithfulness,
                # context_precision,
                # context_recall,
                answer_relevancy_ru,
                # answer_correctness,
                # answer_correctness_ru,
                # faithfulness_ru,
                answer_correctness_devide_ru,
                noise_sensitivity,
                ],
        llm=llm, 
        embeddings=embedder, 
        run_config=config
    )
    return metrics.to_pandas()

In [ ]:
def evaluate_RAG(res_path):
    res = pickle.load(open(res_path, 'rb'))
    dataset = create_dataset(res)
    metrics = evaluate_all_metrics(dataset, llm, config)
    metrics.to_csv(res_path.split('.')[0] + '.csv')

In [ ]:
evaluate_RAG('vanilla.pkl')

In [ ]:
self.embedder_tokenizer = EmbedderTokenizer(self.cfg['retriever']['embedder_tokenizer'])

In [ ]:
pd.read_csv('vanilla.csv')